In [90]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /Users/ddx/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [91]:
import numpy as np
import pandas as pd

import json
import glob


import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim.models import Phrases
from gensim.models.phrases import Phraser

import spacy
from nltk.corpus import stopwords

import pyLDAvis
import pyLDAvis.gensim
import pyLDAvis.gensim_models as gensimvis

import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)



In [92]:
def load_data(file):
    with open(file, 'r', encoding='utf-8') as f:
        data = json.load(f)
    return data

def write_data(file, data):
    with open(file, 'w', encoding='utf-8') as f:
        json.dump(data, f, indent=4)

In [93]:
stopwords = stopwords.words("english")
# stopwords

In [94]:
data = pd.read_csv('data.csv')['Description'].str.replace('About the Creative City:', '', regex=False)

data.head()

0    Situated in a picturesque valley at the foothi...
1     The city of Gangneung is developing an inclus...
2    Casablanca is a major centre for media arts an...
3     Valencia emphasizes the essential role of des...
4     Veliky Novgorod Creative City has made signif...
Name: Description, dtype: object

In [95]:
data[5][0:90]

'\xa0Ouarzazate is considered to have a special place in the minds of film lovers. As one of t'

In [96]:
text = "\xa0Da Lat's economic growth is driven by its creative sector."
length = len(text)
print(length)

59


In [97]:
# !python3 -m spacy download en_core_web_sm
def lemmatization(texts, allowed_postags=["NOUN", "ADJ", "VERB"]):
    nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])
    texts_out = []
    for text in texts:
        doc = nlp(text)
        new_text = []
        for token in doc:
            if token.pos_ in allowed_postags:
                new_text.append(token.lemma_)
        final = " ".join(new_text)
        texts_out.append(final)
    return (texts_out)


lemmatized_texts = lemmatization(data)
print (lemmatized_texts[0][0:90])

situate picturesque valley foothill meeting point ancient civilization unique location inf


In [98]:
def gen_words(texts):
    final = []
    for text in texts:
        new = gensim.utils.simple_preprocess(text, deacc=True)
        final.append(new)
    return (final)

data_words = gen_words(lemmatized_texts)

print (data_words[0][0:20])

['situate', 'picturesque', 'valley', 'foothill', 'meeting', 'point', 'ancient', 'civilization', 'unique', 'location', 'influence', 'city', 'design', 'offer', 'harmonious', 'blend', 'modern', 'technology', 'architectural', 'tradition']


In [99]:
id2word = corpora.Dictionary(data_words)

corpus = []
for text in data_words:
    new = id2word.doc2bow(text)
    corpus.append(new)

print (corpus[0][0:20])

word = id2word[[0][:1][0]]
print (word)

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 2), (6, 1), (7, 1), (8, 1), (9, 2), (10, 1), (11, 2), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 4)]
active


In [100]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=10,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha="auto")

In [101]:
pyLDAvis.enable_notebook()

vis = gensimvis.prepare(lda_model, corpus, id2word, mds="mmds", R=10, n_jobs=1)
vis


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
12    -0.253321 -0.299814       1        1  74.631196
5     -0.091949 -0.467757       2        1   9.861256
7     -0.452397 -0.177989       3        1   9.374077
10     0.221018 -0.370949       4        1   2.072256
21     0.359223 -0.171979       5        1   1.351644
22     0.348007  0.069939       6        1   1.211209
4      0.116597  0.255345       7        1   0.491898
29     0.089272 -0.091833       8        1   0.241576
27    -0.124115  0.058958       9        1   0.183860
1     -0.045202 -0.033813      10        1   0.158375
20    -0.067807  0.116731      11        1   0.114485
8     -0.012138  0.119947      12        1   0.096965
28     0.021277  0.056090      13        1   0.036721
9     -0.006745  0.034958      14        1   0.032723
25     0.000981  0.063283      15        1   0.018671
17    -0.010855  0.063865      16        1   0.017996
3     -0.007905  0.056660      17        1   0.011066
24    -0.007981  0.054775      18        1   0.010920
15    -0.006330  0.055299      19        1   0.007182
19    -0.006330  0.055299      20        1   0.007098
14    -0.006330  0.055299      21        1   0.007060
11    -0.006330  0.055299      22        1   0.007018
2     -0.006330  0.055299      23        1   0.006991
26    -0.006330  0.055299      24        1   0.006919
13    -0.006330  0.055299      25        1   0.006901
16    -0.006330  0.055299      26        1   0.006890
0     -0.006330  0.055299      27        1   0.006884
6     -0.006330  0.055299      28        1   0.006749
23    -0.006330  0.055299      29        1   0.006722
18    -0.006330  0.055299      30        1   0.006693, topic_info=             Term        Freq       Total Category  logprob  loglift
21         design  641.000000  641.000000  Default  30.0000  30.0000
331         music  672.000000  672.000000  Default  29.0000  29.0000
119      cultural  883.000000  883.000000  Default  28.0000  28.0000
45       industry  566.000000  566.000000  Default  27.0000  27.0000
52          local  625.000000  625.000000  Default  26.0000  26.0000
..            ...         ...         ...      ...      ...      ...
160          more    0.000888  180.434121  Topic30  -8.4235  -2.6098
164      organize    0.000888  123.013857  Topic30  -8.4235  -2.2268
266   integration    0.000888   31.074387  Topic30  -8.4235  -0.8509
617          book    0.000888   87.018684  Topic30  -8.4235  -1.8806
69   professional    0.000888  174.241182  Topic30  -8.4235  -2.5749

[1884 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
1966      3  0.959265      able
0         1  0.422859    active
0         2  0.101486    active
0         3  0.473602    active
102       1  0.914841  activity
...     ...       ...       ...
359       3  0.015578      year
101       1  0.871440     young
101       2  0.107019     young
101       3  0.005096     young
101       5  0.010192     young

[808 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[13, 6, 8, 11, 22, 23, 5, 30, 28, 2, 21, 9, 29, 10, 26, 18, 4, 25, 16, 20, 15, 12, 3, 27, 14, 17, 1, 7, 24, 19])

In [108]:
# BIGRAMS AND TRIGRAMS
bigrams_phrases = Phrases(data_words, min_count=5, threshold=50)
trigram_phrases = Phrases(bigrams_phrases[data_words], threshold=50)

# Creating phrasers for efficiency
bigram = Phraser(bigrams_phrases)
trigram = Phraser(trigram_phrases)

# Functions to create bigrams and trigrams
def make_bigrams(texts):
    return [bigram[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram[bigram[doc]] for doc in texts]

# Applying the bigram and trigram models
data_bigrams = make_bigrams(data_words)
data_bigrams_trigrams = make_trigrams(data_bigrams)

# Printing the first document's bigrams/trigrams
print(data_bigrams_trigrams[0])


['situate', 'picturesque', 'valley', 'foothill', 'meeting_point', 'ancient', 'civilization', 'unique', 'location', 'influence', 'city', 'design', 'offer', 'harmonious', 'blend', 'modern', 'technology', 'architectural', 'tradition', 'city', 'centre', 'design', 'renowned', 'innovative', 'project', 'field', 'design', 'public_private', 'sector', 'professional', 'designer', 'play', 'active', 'role', 'today', 'design', 'industry', 'encompass', 'enterprise', 'employ', 'skilled', 'individual', 'showcase', 'commitment', 'urban', 'design', 'host', 'exhibition', 'gather', 'lead', 'local', 'national', 'international', 'designer', 'follower', 'project', 'implement', 'strengthen', 'tie', 'graduate', 'student', 'host', 'various', 'competition', 'design', 'combine', 'traditional', 'contemporary', 'trend', 'project', 'form', 'creative', 'union', 'professional', 'young', 'aspire', 'designer', 'connect', 'municipality', 'community', 'field', 'design', 'dedicate', 'provide', 'training', 'opportunity', 'as

In [109]:
print(data_bigrams_trigrams[0:50])

[['situate', 'picturesque', 'valley', 'foothill', 'meeting_point', 'ancient', 'civilization', 'unique', 'location', 'influence', 'city', 'design', 'offer', 'harmonious', 'blend', 'modern', 'technology', 'architectural', 'tradition', 'city', 'centre', 'design', 'renowned', 'innovative', 'project', 'field', 'design', 'public_private', 'sector', 'professional', 'designer', 'play', 'active', 'role', 'today', 'design', 'industry', 'encompass', 'enterprise', 'employ', 'skilled', 'individual', 'showcase', 'commitment', 'urban', 'design', 'host', 'exhibition', 'gather', 'lead', 'local', 'national', 'international', 'designer', 'follower', 'project', 'implement', 'strengthen', 'tie', 'graduate', 'student', 'host', 'various', 'competition', 'design', 'combine', 'traditional', 'contemporary', 'trend', 'project', 'form', 'creative', 'union', 'professional', 'young', 'aspire', 'designer', 'connect', 'municipality', 'community', 'field', 'design', 'dedicate', 'provide', 'training', 'opportunity', 'a

In [110]:
#TF-IDF REMOVAL
from gensim.models import TfidfModel

id2word = corpora.Dictionary(data_bigrams_trigrams)

texts = data_bigrams_trigrams

corpus = [id2word.doc2bow(text) for text in texts]
# print (corpus[0][0:20])

tfidf = TfidfModel(corpus, id2word=id2word)

low_value = 0.03
words  = []
words_missing_in_tfidf = []
for i in range(0, len(corpus)):
    bow = corpus[i]
    low_value_words = [] #reinitialize to be safe. You can skip this.
    tfidf_ids = [id for id, value in tfidf[bow]]
    bow_ids = [id for id, value in bow]
    low_value_words = [id for id, value in tfidf[bow] if value < low_value]
    drops = low_value_words+words_missing_in_tfidf
    for item in drops:
        words.append(id2word[item])
    words_missing_in_tfidf = [id for id in bow_ids if id not in tfidf_ids] # The words with tf-idf socre 0 will be missing

    new_bow = [b for b in bow if b[0] not in low_value_words and b[0] not in words_missing_in_tfidf]
    corpus[i] = new_bow

In [127]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=7,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha="auto")

In [128]:
pyLDAvis.enable_notebook()

vis = gensimvis.prepare(lda_model, corpus, id2word, mds="mmds", R=10, n_jobs=1)
vis


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
5     -0.074627  0.251733       1        1  68.891181
4     -0.088970 -0.261670       2        1  10.065357
3     -0.262463  0.059924       3        1   7.683409
1      0.206440  0.174892       4        1   5.920561
2      0.244786 -0.152203       5        1   4.611331
0     -0.025166 -0.072677       6        1   2.828161, topic_info=           Term        Freq       Total Category  logprob  loglift
325       music  557.000000  557.000000  Default  10.0000  10.0000
377        film  203.000000  203.000000  Default   9.0000   9.0000
131        food  163.000000  163.000000  Default   8.0000   8.0000
637    literary  170.000000  170.000000  Default   7.0000   7.0000
879       craft  211.000000  211.000000  Default   6.0000   6.0000
..          ...         ...         ...      ...      ...      ...
361      cinema   17.940865   24.682478   Topic6  -4.3020   3.2465
388  production   28.848425  109.760445   Topic6  -3.8270   2.2293
311    festival   14.788807  122.584962   Topic6  -4.4952   1.4506
52     location    9.284773   31.929880   Topic6  -4.9607   2.3304
44     industry   12.301513  225.212517   Topic6  -4.6793   0.6582

[112 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
101       5  0.962861  agriculture
1643      1  0.158336    animation
1643      6  0.791678    animation
294       1  0.484816          art
294       2  0.509057          art
...     ...       ...          ...
352       3  0.035221        world
352       4  0.325795        world
653       4  0.960124        write
654       4  0.988668       writer
938       4  0.964101      writing

[171 rows x 3 columns], R=10, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[6, 5, 4, 2, 3, 1])